### Tech Stack

In [1]:
import bs4
import os

# General 
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage


# Agents
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor



USER_AGENT environment variable not set, consider setting it to identify your requests.


# DONT FORGET TO INCLUDE KEYS 

In [5]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = langchain_key

In [6]:
os.environ['OPENAI_API_KEY'] = openai_key

In [7]:
os.environ['TAVILY_API_KEY'] = tavily_key

In [8]:


llm = ChatOpenAI()

In [9]:
llm.invoke("Motiviere mich in einem Satz die LLM Coding Challenge zu meistern.")

AIMessage(content='Meistere die LLM Coding Challenge, um deine Fähigkeiten im Programmieren zu verbessern und neue berufliche Möglichkeiten zu erschließen.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 25, 'total_tokens': 59, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f9db96c6-312a-4734-93a6-e536d3c3233d-0', usage_metadata={'input_tokens': 25, 'output_tokens': 34, 'total_tokens': 59})

In [10]:
#We can also guide its response with a prompt template. 
#Prompt templates convert raw user input to better input to the LLM.

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class motivator."),
    ("user", "{input}")
])

In [11]:
output_parser = StrOutputParser()

### Query ChatGPT - Build a simple chain

In [12]:
# Build a simple chain
chain = prompt | llm | output_parser

In [13]:
chain.invoke({"input": "Motiviere mich in einem Satz die LLM Coding Challenge zu meistern."})

'Du hast das Potential, die LLM Coding Challenge zu meistern und deine Fähigkeiten auf ein neues Level zu heben - sei fokussiert, bleib dran und lass deinen Ehrgeiz dich zum Erfolg führen!'

### Retrieval Chain
- A retriever is a component used to fetch or retrieve relevant documents or information from a large collection of data
- The retriever’s role is to identify and return the most relevant pieces

In [14]:
# Use Web Page as External Source

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.futurebrains.io/team/nicolas-a-durr")

docs = loader.load()

In [15]:
# Embeddings Model
embeddings = OpenAIEmbeddings()

In [16]:
# Ingest Documents into a Vector Database
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [17]:
# This function generates a chain that takes in several documents (or pieces of text) 
# and merges them into a single response or output.

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


In [18]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrieval_chain.invoke({"input": "Who is Nicolas Dürr?"})
print(response["answer"])


Nicolas A. Dürr is the CEO & Founder of futurebrains, a company that specializes in digital transformation and offers services to startups and businesses. He has experience in startups, the middle market, and enterprises, as well as a background in IT security, IoT, and management.


### Conversation - Modify Chain to answer follow up questions
- Retrieval should consider whole chat history

In [20]:
# Use  of Placeholder  designed to be replaced with actual chat history
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
# Human Message -> User
# AI Message -> Response from LLM
chat_history = [HumanMessage(content="Can Nicolas help me transforming my Business?"), 
                AIMessage(content="Absolutely, he is a Builder by Heart!")]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

print(response["answer"])


Nicolas is a bridge builder between business and technology, leveraging his expertise in IT security, IoT, machine engineering, building management, and mobility. He challenges both leadership and technical teams to implement new business models effectively and establish them in the market.


### Agents - More Power to the LLM 
- so far created examples of chains, where each step is known ahead of time.
- Now let the LLM decide what steps to take next. 

In [23]:
# Give access of tools to the agent


# retriever: o # It could be 
retriever_tool = create_retriever_tool(
    retriever, # object that will be used by the tool to perform searches or retrieve information.
    "nicolas_search", #  identifier for the tool
    "Search for information about Nicolas. For any questions about Nicolas, you must use this tool!", #  provides context or guidance on what the tool is intended
)

# Purpose: This tool helps in integrating search functionality into your system for specific tasks.

"""
Example of using the retriever_tool to perform a search

query = "Tell me about LangSmith's latest updates"
results = retriever_tool.search(query)
print(results)

"""

'\nExample of using the retriever_tool to perform a search\n\nquery = "Tell me about LangSmith\'s latest updates"\nresults = retriever_tool.search(query)\nprint(results)\n\n'

### Use Search Tool Tavily

- Enhance Contextual Awareness: Maintain conversation context across multiple interactions.
- Track Information Sources: Ensure transparency by tracking where information is sourced from.
- Improve User Experience: Provide more engaging and personalized interactions based on user history and preferences.
                

In [24]:
# Use Search Tool Tavily

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [25]:
# Create a list of tools to work with
tools = [retriever_tool, search]

In [26]:
# LangChain Hub provides valuable resources that can help you build and customize agent
# pre-defined prompts, tools, and templates
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

C:\Users\rmai\Python38\lib\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [27]:
chat_history = [HumanMessage(content="Can Nicolas help me transforming my Business?"), 
                AIMessage(content="Absolutely, he is a Builder by Heart!")]

response = agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
#print(response)
#print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `nicolas_search` with `{'query': 'Nicolas business transformation'}`


Nicolas A. Dürr








homeServicesKundenstimmenÜber unsPodcastNEUTermin vereinbarenHomeServicesKundenstimmenÜber unsPodcastNEUtermin vereinbarenWir sind Ihre digitalen SpezialistenNicolas A. DürrCEO & Founder"Die digitale Transformation gewinnt man nicht an der Seitenlinie!"‍‍ÜberNicolasNicolas verfügt über langjährige Erfahrungen bei Startups, im Mittelstand und in Enterprises!Als Entwicklungsleiter bei AXOOM, einem Spin-Off des Maschinenbauers TRUMPF, war er zuletzt für seine vierzigköpfige Entwicklungsableitung tätig und hat eine IoT-Plattform für den Maschinenbau verantwortet.Zuvor hat er bei Sophos Cloud Sicherheitslösungen entwickelt, um Cloud-Umgebungen in AWS und Microsoft Azure abzusichern.Neben seinem Diplom in Informatik des Karlsruher Instituts für Technologie (KIT) hat Nicolas einen Master in Management und Ökonomie der RTPU.Nicolas ist seit mehreren 